In [1]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [10]:
# Load dataset
df = pd.read_csv('C:/Users/Kornelius/Desktop/Data 2/nips-papers/papers.csv', header = 0, sep = ';', error_bad_lines=False)
# Drop the columns not needed
df = df.drop(columns=['id', 'event_type', 'pdf_name'], axis=1)

,year,title,abstract,paper_text
0,2008,A general framework for investigating how far ...,``How is information decoded in the brain?'' i...,A general framework for investigating how far ...
1,2017,MarrNet: 3D Shape Reconstruction via 2.5D Sket...,3D object reconstruction from a single image i...,MarrNet: 3D Shape Reconstruction via 2.5D Sket...
2,2015,Monotone k-Submodular Function Maximization wi...,A $k$-submodular function is a generalization ...,Monotone k-Submodular Function Maximization\nw...
3,2014,The Large Margin Mechanism for Differentially ...,A basic problem in the design of privacy-prese...,The Large Margin Mechanism\nfor Differentially...
4,2011,Bayesian Partitioning of Large-Scale Distance ...,A Bayesian approach to partitioning distance m...,Bayesian Partitioning of Large-Scale Distance ...
5,2016,Linear dynamical neural population models thro...,A body of recent work in modeling neural activ...,Linear dynamical neural population models thro...
6,2012,Bandit Algorithms boost Brain Computer Interfa...,A brain-computer interface (BCI) allows users ...,Bandit Algorithms boost motor-task selection f...
7,2017,Dynamic Routing Between Capsules,A capsule is a group of neurons whose activity...,Dynamic Routing Between Capsules\n\nSara Sabou...
8,2016,Interpretable Nonlinear Dynamic Modeling of Ne...,A central challenge in neuroscience is underst...,Interpretable Nonlinear Dynamic Modeling\nof N...
9,2016,Deep Learning Models of the Retinal Response t...,A central challenge in sensory neuroscience is...,Deep Learning Models of the Retinal Response t...


In [11]:
# Drop a row by condition
df = df[df.abstract != 'Abstract Missing']
# Print out the first rows of papers

,year,title,abstract,paper_text
0,2008,A general framework for investigating how far ...,``How is information decoded in the brain?'' i...,A general framework for investigating how far ...
1,2017,MarrNet: 3D Shape Reconstruction via 2.5D Sket...,3D object reconstruction from a single image i...,MarrNet: 3D Shape Reconstruction via 2.5D Sket...
2,2015,Monotone k-Submodular Function Maximization wi...,A $k$-submodular function is a generalization ...,Monotone k-Submodular Function Maximization\nw...
3,2014,The Large Margin Mechanism for Differentially ...,A basic problem in the design of privacy-prese...,The Large Margin Mechanism\nfor Differentially...
4,2011,Bayesian Partitioning of Large-Scale Distance ...,A Bayesian approach to partitioning distance m...,Bayesian Partitioning of Large-Scale Distance ...
5,2016,Linear dynamical neural population models thro...,A body of recent work in modeling neural activ...,Linear dynamical neural population models thro...
6,2012,Bandit Algorithms boost Brain Computer Interfa...,A brain-computer interface (BCI) allows users ...,Bandit Algorithms boost motor-task selection f...
7,2017,Dynamic Routing Between Capsules,A capsule is a group of neurons whose activity...,Dynamic Routing Between Capsules\n\nSara Sabou...
8,2016,Interpretable Nonlinear Dynamic Modeling of Ne...,A central challenge in neuroscience is underst...,Interpretable Nonlinear Dynamic Modeling\nof N...
9,2016,Deep Learning Models of the Retinal Response t...,A central challenge in sensory neuroscience is...,Deep Learning Models of the Retinal Response t...


In [13]:
data = df.abstract.values.tolist()

In [14]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(data):
    stop_free = " ".join([i for i in data.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(data).split() for data in data]  

In [15]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [16]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(16129 unique tokens: ['activity', 'also', 'analysis', 'applied', 'assume']...)


In [8]:
# filter words that occur in more than 90% of documents
id2word.filter_extremes(no_above = 0.900)
print(id2word)

Dictionary(4415 unique tokens: ['activity', 'also', 'analysis', 'applied', 'assume']...)


In [11]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
lda_model5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [14]:
# Print the Keyword in the 1 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"algorithm" + 0.014*"model" + 0.011*"learning" + 0.011*"problem" + '
  '0.011*"method" + 0.009*"data" + 0.008*"show" + 0.006*"result" + '
  '0.006*"approach" + 0.006*"function"')]


In [15]:
# Print the Keyword in the 5 topics
pprint(lda_model5.print_topics())
doc_lda = lda_model5[corpus]

# Print the Keyword in the 10 topics
pprint(lda_model10.print_topics())
doc_lda = lda_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lda_model20.print_topics())
doc_lda = lda_model20[corpus]

# Print the Keyword in the 50 topics
pprint(lda_model50.print_topics())
doc_lda = lda_model50[corpus]

# Print the Keyword in the 100 topics
pprint(lda_model100.print_topics())
doc_lda = lda_model100[corpus]

[(0,
  '0.026*"learning" + 0.024*"data" + 0.023*"method" + 0.019*"approach" + '
  '0.014*"distribution" + 0.010*"model" + 0.009*"framework" + 0.009*"label" + '
  '0.009*"propose" + 0.008*"task"'),
 (1,
  '0.026*"learning" + 0.023*"algorithm" + 0.017*"online" + 0.016*"regret" + '
  '0.015*"problem" + 0.015*"bound" + 0.013*"setting" + 0.012*"study" + '
  '0.011*"optimal" + 0.010*"policy"'),
 (2,
  '0.050*"image" + 0.022*"network" + 0.022*"deep" + 0.020*"object" + '
  '0.019*"word" + 0.016*"task" + 0.015*"training" + 0.012*"representation" + '
  '0.012*"neural" + 0.011*"translation"'),
 (3,
  '0.062*"model" + 0.015*"network" + 0.013*"structure" + 0.011*"data" + '
  '0.010*"system" + 0.008*"signal" + 0.008*"inference" + 0.007*"memory" + '
  '0.007*"variable" + 0.007*"information"'),
 (4,
  '0.031*"algorithm" + 0.022*"problem" + 0.013*"function" + 0.011*"result" + '
  '0.011*"method" + 0.010*"show" + 0.009*"matrix" + 0.008*"optimization" + '
  '0.008*"bound" + 0.007*"graph"')]
[(0,
  '0.067

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity5: ', lda_model5.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity10: ', lda_model10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity20: ', lda_model20.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity50: ', lda_model50.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity100: ', lda_model100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100: ', coherence_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence ScoreUMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100UMass: ', coherence_lda)


Perplexity:  -7.318990688575403

Perplexity5:  -7.252011130616767

Perplexity10:  -7.273317030403487

Perplexity20:  -7.272599836321585

Perplexity50:  -7.255893799215422

Perplexity100:  -20.95130111001382

Coherence Score:  0.2357683522987768

Coherence Score5:  0.43434360700216124

Coherence Score10:  0.3955419264337291

Coherence Score20:  0.38508875505053064

Coherence Score50:  0.38287359566897883

Coherence Score100:  0.40607945589950356

Coherence ScoreUMass:  -1.4635242209070833

Coherence Score5UMass:  -1.8755717102328464

Coherence Score10UMass:  -3.371076477597785

Coherence Score20UMass:  -5.921739039959454

Coherence Score50UMass:  -8.090264986114605

Coherence Score100UMass:  -10.329427291841258
